In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Sumarização de textos com Generative AI na Vertex AI

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Execute no Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      Veja no GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Execute no Vertex AI Workbench
    </a>
  </td>
</table>

## Visão geral
A sumarização de textos produz um resumo conciso de um texto. Existem dois tipos principais de resumo de texto: extrativo e abstrativo. A sumarização extrativa envolve selecionar frases críticas do texto original e combiná-las para formar um resumo. A sumarização abstrativa envolve a geração de novas sentenças que representam os pontos principais do texto original. Neste notebook, você passará por alguns exemplos de como LLM podem ajudar na geração de sumários de textos.

Saiba mais sobre resumo de texto na [documentação oficial](https://cloud.google.com/vertex-ai/docs/generative-ai/text/summarization-prompts).

### Objetivo

Neste tutorial, você aprenderá como usar LLM para sumarizar informações de texto trabalhando com os seguintes exemplos:
- Sumário da transcrição
- Sumarização de texto em bullets
- Sumário de diálogo com tarefas realizadas
- Tokenização de hashtag
- Geração de títulos e cabeçalhos

Você também aprenderá como avaliar resumos gerados por modelos comparando-os com resumos criados por humanos usando o `ROUGE` como uma estrutura de avaliação.

### Custos
Este tutorial usa os seguintes componentes de Google Cloud:

* Vertex AI Generative AI Studio

Saiba mais sobre possíveis custos envolvidos [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing),
e use a [Calculadora de preços](https://cloud.google.com/products/calculator/)
para gerar uma estimativa de custo com base no uso projetado.

## Primeiros Passos

### Instalando os SDK da Vertex AI e da Cloud Translate API

In [ ]:
!pip install google-cloud-aiplatform google-cloud-translate --upgrade --user

**Somente Colab:** Descomente a célula a seguir para reiniciar o kernel ou use o botão para reiniciar o kernel.

In [ ]:
# # Reinicia automaticamente o kernel após as instalações para que seu ambiente possa acessar os novos pacotes
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Autenticando seu ambiente de notebook
* Se você estiver usando o **Colab** para executar este notebook, descomente a célula abaixo e continue.
* Se você estiver usando o **Vertex AI Workbench**, confira as instruções de configuração [aqui](../setup-env/README.md).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Importando as bibliotecas necessárias

**Somente Colab:** Descomente a célula a seguir para realizar o processo adequado de inicialização da SDK da Vertex AI.  

In [ ]:
# import vertexai

# PROJECT_ID = "[seu-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
from vertexai.preview.language_models import TextGenerationModel

#### Carregando o modelo `text-bison`

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

#### Criando a função *wrapper* para utilizar os modelos em Português

Até o momento desde treinamento, as API do Generative AI Studio suportam somente interações no idioma inglês. Para fazermos interações utilizando o idioma português, vamos utilizar a [Cloud Translation API](https://cloud.google.com/translate) para traduzir as nossas solicitações do português para o inglês e as respostas da API de inglês para português.

In [ ]:
from google.cloud import translate

project_id = !gcloud config list project
project_id = project_id[1].split('=')[1].strip()
parent = f'projects/' + project_id


def traduza(texto, idioma_destino):
    client = translate.TranslationServiceClient()

    response = client.translate_text(
        parent=parent,
        contents=[texto],
        target_language_code=idioma_destino,
        mime_type="text/plain"
    )

    return response.translations[0].translated_text

## Sumarização de texto

### Sumarizar uma transcrição

Neste primeiro exemplo, você resume um texto sobre computação quântica.

In [ ]:
prompt = traduza("""
Proporcione un resumen muy breve, no más de tres oraciones, para el siguiente artículo:

Nuestras computadoras cuánticas funcionan manipulando qubits de una manera orquestada que llamamos algoritmos cuánticos.
El desafío es que los qubits son tan sensibles que incluso la luz dispersa puede causar errores de cálculo, y el problema empeora a medida que crecen las computadoras cuánticas.
Esto tiene consecuencias significativas, ya que los mejores algoritmos cuánticos que conocemos para ejecutar aplicaciones útiles requieren que las tasas de error de nuestros qubits sean mucho más bajas que las que tenemos hoy.
Para llenar este vacío, necesitaremos la corrección del error cuántico.
La corrección de errores cuánticos protege la información al codificarla en múltiples qubits físicos para formar un "qubit lógico" y se cree que es la única forma de producir una computadora cuántica a gran escala con tasas de error lo suficientemente bajas como para realizar cálculos útiles.
En lugar de calcular en los qubits individuales, calcularemos en qubits lógicos. Al codificar un mayor número de qubits físicos en nuestro procesador cuántico en un qubit lógico, esperamos reducir las tasas de error para permitir algoritmos cuánticos útiles.

Resumen:

""", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8).text, "es")
)

Em vez de um resumo, podemos pedir um TL;DR (*"too long; didn't read"* ou "muito longo; não li" em tradução livre). Você pode comparar as diferenças entre as saídas geradas.

In [ ]:
prompt = traduza("""
Proporcione un TL;DR para el siguiente artículo:

Nuestras computadoras cuánticas funcionan manipulando qubits de una manera orquestada que llamamos algoritmos cuánticos.
El desafío es que los qubits son tan sensibles que incluso la luz dispersa puede causar errores de cálculo, y el problema empeora a medida que crecen las computadoras cuánticas.
Esto tiene consecuencias significativas, ya que los mejores algoritmos cuánticos que conocemos para ejecutar aplicaciones útiles requieren que las tasas de error de nuestros qubits sean mucho más bajas que las que tenemos hoy.
Para llenar este vacío, necesitaremos la corrección del error cuántico.
La corrección de errores cuánticos protege la información al codificarla en múltiples qubits físicos para formar un "qubit lógico" y se cree que es la única forma de producir una computadora cuántica a gran escala con tasas de error lo suficientemente bajas como para realizar cálculos útiles.
En lugar de calcular en los qubits individuales, calcularemos en qubits lógicos. Al codificar un mayor número de qubits físicos en nuestro procesador cuántico en un qubit lógico, esperamos reducir las tasas de error para permitir algoritmos cuánticos útiles.

TL;DR:
""", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8).text, "es")
)

### Sumarizar em bullets
No exemplo a seguir, você usa o mesmo texto sobre computação quântica, mas pede ao modelo para sumarizar com bullets. Sinta-se à vontade para alterar o prompt.

In [ ]:
prompt = traduza("""
Proporcione un resumen muy breve en cuatro bullets para el siguiente artículo:

Nuestras computadoras cuánticas funcionan manipulando qubits de una manera orquestada que llamamos algoritmos cuánticos.
El desafío es que los qubits son tan sensibles que incluso la luz dispersa puede causar errores de cálculo, y el problema empeora a medida que crecen las computadoras cuánticas.
Esto tiene consecuencias significativas, ya que los mejores algoritmos cuánticos que conocemos para ejecutar aplicaciones útiles requieren que las tasas de error de nuestros qubits sean mucho más bajas que las que tenemos hoy.
Para llenar este vacío, necesitaremos la corrección del error cuántico.
La corrección de errores cuánticos protege la información al codificarla en múltiples qubits físicos para formar un "qubit lógico" y se cree que es la única forma de producir una computadora cuántica a gran escala con tasas de error lo suficientemente bajas como para realizar cálculos útiles.
En lugar de calcular en los qubits individuales, calcularemos en qubits lógicos. Al codificar un mayor número de qubits físicos en nuestro procesador cuántico en un qubit lógico, esperamos reducir las tasas de error para permitir algoritmos cuánticos útiles.

Bullets:

""", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=256, top_k=1, top_p=0.8).text, "es")
)

### Sumarizar diálogos com tarefas realizadas
O sumário do diálogo envolve condensar uma conversa em um formato mais curto para que você não precise ler toda a discussão, mas possa aproveitar um resumo. Neste exemplo, você pede ao modelo para resumir uma conversa de exemplo entre um cliente de varejo online e um agente de suporte e incluir tarefas no final.

In [ ]:
prompt = traduza("""
Genere un resumen de la siguiente conversación y, al final, resuma las tareas para el agente de soporte:

Cliente: Hola, soy Joe y recibí el artículo equivocado.

Agente de soporte: Hola José. ¿Cómo le gustaría ver esto resuelto?

Cliente: Quiero devolver el artículo y obtener un reembolso, por favor.

Agente de apoyo: Por supuesto. Puedo procesar el reembolso por usted ahora. ¿Puedo tener su número de orden por favor?

Cliente: Es [NÚMERO DE PEDIDO].

Agente de soporte: Gracias. He procesado el reembolso y te devolveremos el dinero en un plazo de 14 días.

Cliente: Muchas gracias.

Agente de apoyo: De nada, José. ¡Tenga un buen día!

Resumen:
""", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=256, top_k=40, top_p=0.8).text, "es")
)

### Tokenização de hashtag
A tokenização de hashtag é o processo de pegar um pedaço de texto e obter os "tokens" de hashtag. Você pode usar isso, por exemplo, se quiser gerar hashtags para suas campanhas de mídia social. Neste exemplo, você pega [este tweet do Google Cloud](https://twitter.com/googlecloud/status/1649127992348606469) e gera algumas hashtags que pode usar.

In [ ]:
prompt = traduza("""
Tokenizar los hashtags de este tuit:

Nube de Google
@googlecloud
¿Cómo pueden los datos ayudar a nuestro planeta cambiante? 🌎

En honor al #DíaDeLaTierra este fin de semana, nos enorgullece compartir cómo nos asociamos con
@ClimateEngine para aprovechar el poder de los datos geoespaciales e impulsar hacia un futuro más sostenible.

Mira cómo → https://goo.gle/3mOUfts
""", "en")

print(
    generation_model.predict(prompt, temperature=0.8, max_output_tokens=1024, top_k=40, top_p=0.8).text
)

### Geração de títulos e cabeçalhos
Abaixo, você pede ao modelo para gerar cinco opções para possíveis combinações de título/cabeçalho para um determinado trecho de texto.

In [ ]:
prompt = traduza("""
Escribe un título para este texto, dame cinco opciones:
Ya sea para ayudar a los médicos a identificar enfermedades o encontrar fotos de "abrazos", la IA está detrás de gran parte del trabajo que hacemos en Google. Y en nuestro Laboratorio de Arte y Cultura en París, hemos estado experimentando cómo se puede usar la IA para beneficiar la cultura.
Hoy compartimos nuestros últimos experimentos: prototipos que se basan en siete años de trabajo en asociación con 1500 instituciones culturales de todo el mundo.
Cada una de estas aplicaciones experimentales ejecuta algoritmos de inteligencia artificial en segundo plano para permitirle descubrir conexiones culturales ocultas en archivos e incluso encontrar obras de arte que coincidan con la decoración de su hogar".
""", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.8, max_output_tokens=256, top_k=1, top_p=0.8).text, "es")
)

## Avaliação
Você pode avaliar os resultados das tarefas de resumo usando [ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)) como uma estrutura de avaliação. `ROUGE` (Recall-Oriented Understudy for Gisting Evaluation) são medidas para determinar automaticamente a qualidade de um resumo comparando-o com outros resumos (ideais) criados por humanos. As medidas contam o número de unidades sobrepostas, como n-gram, sequências de palavras e pares de palavras entre o resumo gerado por computador a ser avaliado e os resumos ideais criados por humanos.


O primeiro passo é instalar a biblioteca `ROUGE`.

In [ ]:
!pip install rouge

Crie um resumo a partir de um LLM que você pode usar para comparar com um resumo gerado por humanos.

In [ ]:
from rouge import Rouge

ROUGE = Rouge()

prompt = traduza("""
Proporcione un resumen muy breve, de cuatro oraciones como máximo, para el siguiente artículo:

Nuestras computadoras cuánticas funcionan manipulando qubits de una manera orquestada que llamamos algoritmos cuánticos.
El desafío es que los qubits son tan sensibles que incluso la luz dispersa puede causar errores de cálculo, y el problema empeora a medida que crecen las computadoras cuánticas.
Esto tiene consecuencias significativas, ya que los mejores algoritmos cuánticos que conocemos para ejecutar aplicaciones útiles requieren que las tasas de error de nuestros qubits sean mucho más bajas que las que tenemos hoy.
Para llenar este vacío, necesitaremos la corrección del error cuántico.
La corrección de errores cuánticos protege la información al codificarla en múltiples qubits físicos para formar un "qubit lógico" y se cree que es la única forma de producir una computadora cuántica a gran escala con tasas de error lo suficientemente bajas como para realizar cálculos útiles.
En lugar de calcular en los qubits individuales, calcularemos en qubits lógicos. Al codificar un mayor número de qubits físicos en nuestro procesador cuántico en un qubit lógico, esperamos reducir las tasas de error para permitir algoritmos cuánticos útiles.

Resumen:

""", "en")

candidate = traduza(generation_model.predict(prompt, temperature=0.1, max_output_tokens=1024, top_k=40, top_p=0.9).text, "es")

print(candidate)

You will also need a human-generated summary that we will use to compare to the `candidate` generated by the model. We will call this `reference`. 

In [ ]:
reference = "Las computadoras cuánticas son sensibles al ruido y los errores. Para llenar este vacío, necesitaremos la corrección de errores cuánticos. La corrección de errores cuánticos protege la información mediante la codificación de múltiples qubits físicos para formar un 'qubit lógico'."

Agora você pode pegar o candidato e a referência para avaliar o desempenho. Neste caso, ROUGE lhe dará:

- `rouge-1`, que mede a sobreposição de unigramas
- `rouge-2`, que mede a sobreposição de bigramas
- `rouge-l`, que mede a maior subsequência comum

In [ ]:
ROUGE.get_scores(candidate, reference)